# 08 Exercise CIFAR10 Regularization

In [1]:
# Import librerie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

# Scikit-learn per preprocessing e metriche
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Impostazioni
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Device: {device}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")

import os, urllib.request

# GitHub Release URL for pretrained weights
WEIGHTS_BASE_URL = os.environ.get('WEIGHTS_URL', 'https://github.com/SamueleBolotta/CEAR/releases/download/v1.0/')
WEIGHTS_DIR = '../pretrained_weights'
os.makedirs(WEIGHTS_DIR, exist_ok=True)

def load_or_train(model, train_fn, weights_filename, device='cpu'):
    """Load pretrained weights if available, otherwise train and save.
    Also saves/loads training history as JSON alongside weights."""
    weights_path = os.path.join(WEIGHTS_DIR, weights_filename)
    history_path = weights_path.replace('.pt', '_history.json')

    def _load_history():
        if os.path.exists(history_path):
            import json as _json
            with open(history_path, 'r') as f:
                return _json.load(f)
        return None

    if os.path.exists(weights_path):
        model.load_state_dict(torch.load(weights_path, map_location=device, weights_only=True))
        print(f"Loaded pretrained weights from {weights_path}")
        return _load_history()
    elif WEIGHTS_BASE_URL:
        try:
            url = WEIGHTS_BASE_URL + weights_filename
            urllib.request.urlretrieve(url, weights_path)
            # Also try downloading history
            try:
                urllib.request.urlretrieve(
                    WEIGHTS_BASE_URL + weights_filename.replace('.pt', '_history.json'), history_path)
            except Exception:
                pass
            model.load_state_dict(torch.load(weights_path, map_location=device, weights_only=True))
            print(f"Downloaded and loaded weights from {url}")
            return _load_history()
        except Exception as e:
            print(f"Could not download weights: {e}. Training from scratch...")

    history = train_fn()
    torch.save(model.state_dict(), weights_path)
    print(f"Saved weights to {weights_path}")
    if history is not None:
        import json as _json
        with open(history_path, 'w') as f:
            _json.dump(history, f)
        print(f"Saved training history to {history_path}")
    return history

PyTorch version: 2.10.0+cu128
Torchvision version: 0.25.0+cu128
Device: cuda
CUDA disponibile: True


## Esercizio 3

In [17]:
# ============================================================================
# ESERCIZIO 3: Tecniche di Regolarizzazione per Ridurre Overfitting
# ============================================================================
# Task: Confrontare Dropout, L2 Regularization e Batch Normalization
# Dataset: CIFAR-10 subset con 2 classi (10000 campioni)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from sklearn.model_selection import train_test_split

# Caricamento subset CIFAR-10: solo classi 0 (airplane) e 1 (automobile)
np.random.seed(789)
torch.manual_seed(789)
cifar_train = torchvision.datasets.CIFAR10(root='../data', train=True, download=True)
cifar_test = torchvision.datasets.CIFAR10(root='../data', train=False, download=True)

X_cifar = cifar_train.data  # (50000, 32, 32, 3)
y_cifar = np.array(cifar_train.targets)
X_test_cifar = cifar_test.data
y_test_cifar = np.array(cifar_test.targets)

raise NotImplementedError()

X_train_cifar = X_cifar[mask_train]
y_train_cifar = y_cifar[mask_train]
X_test_cifar_sub = X_test_cifar[mask_test]
y_test_cifar_sub = y_test_cifar[mask_test]

raise NotImplementedError()

print(f"Dataset CIFAR-10 Binario (Airplane vs Automobile)")
print(f"Train: {X_train_cifar.shape}, Test: {X_test_cifar_sub.shape}")
print(f"Distribuzione train: Airplane={np.sum(y_train_cifar==0)}, Auto={np.sum(y_train_cifar==1)}")

# Helper function for training binary CIFAR models with early stopping
def train_binary_cifar(model, X_train_np, y_train_np, epochs=20, batch_size=128, val_split=0.2, patience=3, weight_decay=0.0):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)

    X_t = torch.FloatTensor(X_train_np).to(device)
    y_t = torch.FloatTensor(y_train_np).to(device)

    n_val = int(len(X_t) * val_split)
    idx = torch.randperm(len(X_t))
    X_tr, y_tr = X_t[idx[n_val:]], y_t[idx[n_val:]]
    X_vl, y_vl = X_t[idx[:n_val]], y_t[idx[:n_val]]

    train_ds = TensorDataset(X_tr, y_tr)
    loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}
    best_val_loss = float('inf')
    patience_counter = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for bx, by in loader:
            optimizer.zero_grad()
            out = model(bx).squeeze()
            loss = criterion(out, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * bx.size(0)
            predicted = (torch.sigmoid(out) > 0.5).float()
            total += by.size(0)
            correct += (predicted == by).sum().item()

        train_loss = running_loss / total
        train_acc = correct / total

        model.eval()
        with torch.no_grad():
            val_out = model(X_vl).squeeze()
            val_loss_val = criterion(val_out, y_vl).item()
            val_pred = (torch.sigmoid(val_out) > 0.5).float()
            val_acc = (val_pred == y_vl).sum().item() / len(y_vl)

        history['loss'].append(train_loss)
        history['accuracy'].append(train_acc)
        history['val_loss'].append(val_loss_val)
        history['val_accuracy'].append(val_acc)

        if val_loss_val < best_val_loss:
            best_val_loss = val_loss_val
            patience_counter = 0
            best_state = {k: v.clone() for k, v in model.state_dict().items()}
        else:
            patience_counter += 1
            if patience_counter >= patience:
                model.load_state_dict(best_state)
                break

    return history

def evaluate_binary_cifar(model, X_np, y_np):
    model.eval()
    with torch.no_grad():
        X_t = torch.FloatTensor(X_np).to(device)
        out = model(X_t).squeeze()
        raise NotImplementedError()
        acc = (pred == y_np).mean()
    return acc

# Step 1: Modello baseline (senza regolarizzazione)
model_baseline_c = nn.Sequential(
    nn.Linear(3072, 256), nn.ReLU(),
    nn.Linear(256, 128), nn.ReLU(),
    nn.Linear(128, 1)
).to(device)

print("Modello Baseline:")
print(model_baseline_c)
print(f'Parametri: {sum(p.numel() for p in model_baseline_c.parameters()):,}')

# Step 2: Modello con Dropout
model_dropout_c = nn.Sequential(
    nn.Linear(3072, 256), nn.ReLU(), nn.Dropout(0.4),
    nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.4),
    nn.Linear(128, 1)
).to(device)

# Step 3: Modello con L2 Regularization (weight_decay nell'optimizer)
model_l2_c = nn.Sequential(
    nn.Linear(3072, 256), nn.ReLU(),
    nn.Linear(256, 128), nn.ReLU(),
    nn.Linear(128, 1)
).to(device)

# Step 4: Modello con Batch Normalization
model_bn_c = nn.Sequential(
    nn.Linear(3072, 256), nn.BatchNorm1d(256), nn.ReLU(),
    nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
    nn.Linear(128, 1)
).to(device)

# Training tutti i modelli (with pretrained weight support)
models_dict_c = {
    'Baseline': (model_baseline_c, 0.0, 'nb04_ex3_baseline.pt'),
    'Dropout': (model_dropout_c, 0.0, 'nb04_ex3_dropout.pt'),
    'L2 Regularization': (model_l2_c, 0.01, 'nb04_ex3_l2.pt'),
    'Batch Normalization': (model_bn_c, 0.0, 'nb04_ex3_batchnorm.pt')
}

histories_c = {}
results_c = []

print("\n" + "="*70)
print("TRAINING MODELLI")
print("="*70)

for name, (model_c, wd, wf) in models_dict_c.items():
    print(f'\nTraining {name}...')
    history = load_or_train(
        model_c,
        lambda m=model_c, w=wd: train_binary_cifar(m, X_train_cifar, y_train_cifar, epochs=20, weight_decay=w),
        wf,
        device=device,
    )
    histories_c[name] = history

    test_acc = evaluate_binary_cifar(model_c, X_test_cifar_sub, y_test_cifar_sub)

    if history is not None:
        train_acc = max(history['accuracy'])
        val_acc = max(history['val_accuracy'])
        overfitting_gap = train_acc - val_acc
        n_epochs = len(history['loss'])
    else:
        train_acc = evaluate_binary_cifar(model_c, X_train_cifar, y_train_cifar)
        val_acc = None  # no validation data available with pretrained weights
        overfitting_gap = None
        n_epochs = 'N/A'

    results_c.append({
        'modello': name,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc,
        'overfitting_gap': overfitting_gap,
        'epochs': n_epochs
    })

    print(f'{name}: Test Acc={test_acc:.4f}, Overfitting Gap={f"{overfitting_gap:.4f}" if overfitting_gap is not None else "N/A (pretrained)"}')

results_df_c = pd.DataFrame(results_c).sort_values('test_acc', ascending=False)

print("\n" + "="*70)
print("CONFRONTO TECNICHE REGOLARIZZAZIONE")
print("="*70)
print(results_df_c.to_string(index=False))

# Step 5: Visualizzazione learning curves
if any(h is not None for h in histories_c.values()):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()

    for idx, (name, history) in enumerate(histories_c.items()):
        ax = axes[idx]
        if history is not None:
            ax.plot(history['accuracy'], label='Train')
            ax.plot(history['val_accuracy'], label='Validation')
        else:
            ax.text(0.5, 0.5, 'Pretrained\n(no curves)',
                    ha='center', va='center',
                    transform=ax.transAxes, fontsize=12)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.set_title(f'{name}')
        ax.legend()
        ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print("Using pretrained weights - learning curves not available")

# Confronto overfitting gap
has_gaps = any(g is not None for g in results_df_c['overfitting_gap'].values)
if has_gaps:
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_df = results_df_c[results_df_c['overfitting_gap'].notna()]
    models_names = plot_df['modello'].values
    gaps = plot_df['overfitting_gap'].values.astype(float)
    colors = ['red' if gap > 0.05 else 'green' for gap in gaps]

    ax.barh(models_names, gaps, color=colors, alpha=0.7)
    ax.axvline(x=0.05, color='orange', linestyle='--', label='Soglia Overfitting')
    ax.set_xlabel('Overfitting Gap (Train Acc - Val Acc)')
    ax.set_title('Confronto Overfitting: Tecniche di Regolarizzazione')
    ax.legend()
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Overfitting gap non disponibile con pesi pretrained (servono dati di validazione)")

best_model_name_c = results_df_c.iloc[0]['modello']
print(f'\nMigliore tecnica: {best_model_name_c}')
print(f"Test Accuracy: {results_df_c.iloc[0]['test_acc']:.4f}")
og = results_df_c.iloc[0]['overfitting_gap']
print(f"Overfitting Gap: {f'{og:.4f}' if og is not None else 'N/A (pretrained)'}")

print("\nEsercizio 3 completato!")

# Ripristino seed globali
np.random.seed(42)
_ = torch.manual_seed(42)

100%|██████████| 170M/170M [00:04<00:00, 40.8MB/s]


Dataset CIFAR-10 Binario (Airplane vs Automobile)
Train: (10000, 3072), Test: (2000, 3072)
Distribuzione train: Airplane=5000, Auto=5000
Modello Baseline:
Sequential(
  (0): Linear(in_features=3072, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=1, bias=True)
)
Parametri: 819,713

TRAINING MODELLI

Training Baseline...
Downloaded and loaded weights from https://github.com/SamueleBolotta/CEAR/releases/download/v1.0/nb04_ex3_baseline.pt
Baseline: Test Acc=0.8635, Overfitting Gap=N/A (pretrained)

Training Dropout...
Downloaded and loaded weights from https://github.com/SamueleBolotta/CEAR/releases/download/v1.0/nb04_ex3_dropout.pt
Dropout: Test Acc=0.8575, Overfitting Gap=N/A (pretrained)

Training L2 Regularization...
Downloaded and loaded weights from https://github.com/SamueleBolotta/CEAR/releases/download/v1.0/nb04_ex3_l2.pt
L2 Regularization: Test Acc=0.8515, Overfitting